In [ ]:
#Hein Linn Htet CMSC 206 Individual Project 5

# Cell 1: Import core libraries
import os
import requests
import pandas as pd
import simplekml
from io import StringIO
from pyproj import Geod



In [ ]:
# Cell 2: Report cwd (where the KML will be saved)
cwd = os.getcwd()
print("Working directory:", cwd)
geod = Geod(ellps="WGS84")


Working directory: c:\Users\heinl\OneDrive\Documents\montgomerycollege\CMSC206\01\individualproject5


In [ ]:
# Cell 3: Download the USGS CSV and prepare DataFrame & ring settings
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.csv"
r = requests.get(url)
r.raise_for_status()


from io import StringIO
df = pd.read_csv(StringIO(r.text))


df = df.dropna(subset=["latitude","longitude","mag","place","time"])
df.head(3)

km_per_mag = 10.0

circle_points = 36




In [ ]:
# Cell 4: Build the KML with placemarks + scaled geodesic rings
kml = simplekml.Kml()

for _, row in df.iterrows():
    lon, lat, mag = row["longitude"], row["latitude"], row["mag"]
    place = row["place"]
    time = row["time"]


    desc = f"<h1>{mag} Magnitude earthquake at {time}</h1>"
    kml.newpoint(
        name=place,
        description=desc,
        coords=[(lon, lat)]
    )

    radius_m = mag * km_per_mag * 1000 
    azimuths = list(range(0, 360, int(360/circle_points)))
    circle_coords = []
    for az in azimuths:
        lon2, lat2, _ = geod.fwd(lon, lat, az, radius_m)
        circle_coords.append((lon2, lat2))
    circle_coords.append(circle_coords[0])  

    poly = kml.newpolygon(
        name=f"{mag}-mag ring @ {place}",
        outerboundaryis=circle_coords
    )
    poly.style.polystyle.fill = 0             
    poly.style.linestyle.width = 2            
    poly.style.linestyle.color = "7fff0000"    





In [32]:
# Cell 5: Save to the notebook’s directory
outfile = os.path.join(cwd, "earthquakes_with_rings.kml")
kml.save(outfile)
print(f" Saved KML : {outfile}")


 Saved KML : c:\Users\heinl\OneDrive\Documents\montgomerycollege\CMSC206\01\individualproject5\earthquakes_with_rings.kml
